In [1]:
import csv
import math
import random

In [21]:
 def loadcsv(filename):
        lines = csv.reader(open(r'pima-indians-diabetes.data.csv'))
        dataset = list(lines)
        for i in range(len(dataset)):
            dataset[i] = [float(x) for x in dataset[i]]
        return dataset

In [22]:
#spiliting the dataset
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset)*splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [23]:
# now moving ahead to make the summary of dataset
#seprating data by class, cal mean, st. deviation etc 
# map of each class value to a list of instances that belong to the
# class and sort the entire dataset of instance into the appropriate list

def seprateByClass(dataset):
    # fn assumes that the last attribute is the class value
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated

In [24]:
# need to cal mean of each attribute for a class value
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [25]:
def stdDev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [26]:
def summarize(dataset):
    summaries = [(mean(attribute), stdDev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [27]:
def summariesByClass(dataset):
    separated = seprateByClass(dataset)
    summaries={}
    for classValue, instances in separated.items():
        summaries[classValue]=summarize(instances)
    return summaries

In [28]:
def calProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [29]:
def calClassProbability(summaries, inputVector):
    probabilities= {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue]=1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x= inputVector[i]
            probabilities[classValue]*= calProbability(x, mean, stdev)
        return probabilities

In [30]:
def predict(summaries, inputVector):
    probabilities = calClassProbability(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [31]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [32]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100

In [33]:
def main():
    filename = 'pima-indians-diabetes.data.csv'
    splitRatio= 0.67
    dataset = loadcsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train = {1} and test = {2} rows'.format(len(dataset),len(trainingSet),len(testSet)))
    #prepare model
    summaries = summariesByClass(trainingSet)
    #test model
    predictions = getPredictions(summaries, testSet)
    accuracy= getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))

main()

Split 768 rows into train = 514 and test = 254 rows
Accuracy: 68.11023622047244%
